In [2]:
pip install elasticsearch

                                              0.0/387.9 kB ? eta -:--:--
                                              0.0/387.9 kB ? eta -:--:--
     --------                                81.9/387.9 kB 2.2 MB/s eta 0:00:01
     --------------------                   204.8/387.9 kB 2.1 MB/s eta 0:00:01
     -------------------------              256.0/387.9 kB 1.7 MB/s eta 0:00:01
     ---------------------------            276.5/387.9 kB 1.5 MB/s eta 0:00:01
     ------------------------------         307.2/387.9 kB 1.4 MB/s eta 0:00:01
     ---------------------------------      337.9/387.9 kB 1.2 MB/s eta 0:00:01
     -------------------------------------  378.9/387.9 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 387.9/387.9 kB 1.1 MB/s eta 0:00:00
                                              0.0/59.5 kB ? eta -:--:--
     --------------------------             41.0/59.5 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 59.5/59.5 kB 794.2


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: C:\Users\XDavi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [59]:
from elasticsearch import Elasticsearch
import json
import os
import pandas as pd

# Create the client instance
client = Elasticsearch("http://localhost:9200")

# Successful response!
client.info()

C:\Users\XDavi\AppData\Local\Temp\ipykernel_4628\2156522912.py:10: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.info()


ObjectApiResponse({'name': '9d9a12f07c74', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'FITIbcYeR4erqq6Km1jxLQ', 'version': {'number': '7.17.9', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'ef48222227ee6b9e70e502f0f0daa52435ee634d', 'build_date': '2023-01-31T05:34:43.305517834Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [91]:
# NOTE: THe following code must be run just once
# Get the documents from the directory
entries = os.listdir('1-publications/json')
for i in entries:
    # Opening JSON file
    f = open('1-publications/json/'+i)

    # returns JSON object as a dictionary
    data = json.load(f)
    
    # Closing file
    f.close()

    # Remove the unnecessary fields
    list = ['url','paperId','year','publicationDate','authors']
    for i in list:
        data.pop(i, None)
    client.index(
    index='prueba',
    document=data)

# Refresh the index
client.indices.refresh(index='lord-of-the-rings')

C:\Users\XDavi\AppData\Local\Temp\ipykernel_4628\3784935029.py:18: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.index(
C:\Users\XDavi\AppData\Local\Temp\ipykernel_4628\3784935029.py:23: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.indices.refresh(index='lord-of-the-rings')


ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [94]:
# Query for the keyword you want
val = input("Enter the keyword you are looking for: ")
result = client.search(
 index='prueba',
  query={
    'match': {'abstract': val}
  }
 )
a = result['hits']['hits']
df = pd.DataFrame(columns=['title', 'abstract','score'])
for i in a:
    df = df.append({'title': i['_source']['title'], 'abstract': i['_source']['abstract'],'score':i['_score']}, ignore_index=True)
df

C:\Users\XDavi\AppData\Local\Temp\ipykernel_4628\2039512892.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = client.search(
C:\Users\XDavi\AppData\Local\Temp\ipykernel_4628\2039512892.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': i['_source']['title'], 'abstract': i['_source']['abstract'],'score':i['_score']}, ignore_index=True)
C:\Users\XDavi\AppData\Local\Temp\ipykernel_4628\2039512892.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': i['_source']['title'], 'abstract': i['_source']['abstract'],'score':i['_score']}, ignore_ind

,title,abstract,score
0,Overview of Big Data and Its Visualization,Big Data is data sets that are so voluminous a...,0.170287
1,"Comparison between Expert Systems, Machine Lea...","Today, the science of artificial intelligence ...",0.168505
2,Skin Cancer Diagnosis Using an Improved Ensemb...,In recent years skin cancer is becoming more a...,0.165958
3,"An Overview of Drugs, Diseases, Genes and Prot...",Several initiatives have emerged during the CO...,0.163347
4,"Soluble ST2, a biomarker of fibrosis, is assoc...",Abstract Several diseases have a deleterious f...,0.155209


In [89]:
# DO NOT RUN THIS CODE IF YOU DON'T WANT TO DELETE THE INDEX!!!
client.options(ignore_status=[400,404]).indices.delete(index='prueba')

C:\Users\XDavi\AppData\Local\Temp\ipykernel_4628\2243560813.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.options(ignore_status=[400,404]).indices.delete(index='prueba')


ObjectApiResponse({'acknowledged': True})